In [1]:
from pandas import read_csv
from matplotlib import pyplot
import sys
import numpy as np
import pandas as pd

N=9
nowcast=1

z_input_size=1
y_input_size=1

file="solartech_segmented.csv"

series=read_csv(file, sep=",", header=0,parse_dates=True)
series=series.values
series


array([[0, '2017-06-13 00:00:00', 0.0, 24.851, 0.0],
       [1, '2017-06-13 00:10:00', 0.0, 24.802, 0.0],
       [2, '2017-06-13 00:20:00', 0.0, 24.758, 0.0],
       ...,
       [17133, '2017-11-21 23:30:00', 0.0, 7.949, 0.0],
       [17134, '2017-11-21 23:40:00', 0.0, 7.861, 0.0],
       [17135, '2017-11-21 23:50:00', 0.0, 7.681, 0.0]], dtype=object)

In [2]:

X1=sensors=np.array(series[:,2])
X2=sensors=np.array(series[:,3])
Y0=sensors=np.array(series[:,4])

print(X1[0])
print(X2[0])
print(Y0[0])

0.0
24.851
0.0


In [3]:
import tensorflow as tf 

X1n=tf.keras.utils.normalize(    X1, axis=-1, order=2)
X1n=np.array(X1n[0])
#X1n=X1


print(X1n)
print(X1n.shape)

X2n=tf.keras.utils.normalize(X2, axis=-1, order=2)
X2n=np.array(X2n[0])
#X2n=X2
print(X2n)
print(X2n.shape)

X0=np.zeros((len(X1n),2))

X0[:,0]=X1n
X0[:,1]=X2n

Y=np.zeros((len(Y0),1))
Y[:,0]=Y0

print(X0)
print(X0.shape)
print(Y.shape)
print(Y)

[0.0 0.0 0.0 ... 0.0 0.0 0.0]
(17136,)
[0.008708218143262458 0.008691047699858978 0.00867562934251708 ...
 0.002785466420699098 0.0027546297060152985 0.0026915546077984362]
(17136,)
[[0.         0.00870822]
 [0.         0.00869105]
 [0.         0.00867563]
 ...
 [0.         0.00278547]
 [0.         0.00275463]
 [0.         0.00269155]]
(17136, 2)
(17136, 1)
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [4]:
X=[]
for i in range(N,len(X0)):
    X.append(X0[i-N:i])
X=np.array(X)    
print(X.shape)

Y=Y[N-nowcast:len(Y)-nowcast]
print(Y.shape)



from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
print(Y[40])
scaler.fit(Y)

Y = scaler.transform(Y)
print(Y[40])

(17127, 9, 2)
(17127, 1)
[98.72755432]
[0.40682649]


In [5]:
print(X[0])
print(Y[0])

[[0.         0.00870822]
 [0.         0.00869105]
 [0.         0.00867563]
 [0.         0.00867878]
 [0.         0.008649  ]
 [0.         0.00857646]
 [0.         0.00851269]
 [0.         0.00850287]
 [0.         0.00851304]]
[0.]


In [6]:
from sklearn.model_selection import train_test_split
x_train , x_val , y_train , y_val = train_test_split(X , Y ,test_size=0.2, random_state=0)

In [7]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense,concatenate

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D
from tensorflow.keras.models import Model



def make_generator():    
    x = Input(shape=(N,2), dtype='float')
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)

    noise = Input(shape=(z_input_size,))
    noise_output = Dense(64, activation='relu')(noise)

    concat = concatenate([x_output, noise_output])
    output = Dense(256)(concat)
    output = Dense(128)(output)
    output = Dense(y_input_size, activation="linear")(output)

    model = Model(inputs=[noise, x], outputs=output)
#    model.compile(loss='mean_squared_error', optimizer='rmsprop')        
    model.summary()
    return model

def make_discriminator():
    x = Input(shape=(N,2), dtype='float')
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)

    label = Input(shape=(y_input_size,))
    label_output = Dense(64, activation='relu')(label)

    concat = concatenate([x_output, label_output])
    concat = Dense(256)(concat)
    concat = Dense(128)(concat)
    validity = Dense(1,activation="sigmoid")(concat)

    model = Model(inputs=[x, label], outputs=validity)
    model.compile(loss=['binary_crossentropy'],optimizer='rmsprop',metrics=['accuracy'])
    model.summary()
    return model   



discriminator = make_discriminator()
generator = make_generator()

noise = Input(shape=(z_input_size,))

x = Input(shape=(N,2))
label = generator([noise, x])

discriminator.trainable = False
validity = discriminator([x, label])

combined = Model([noise, x], validity)
combined.compile(loss=['binary_crossentropy'],optimizer='rmsprop')
combined.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 9, 2)]       0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 8, 16)        80          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 6, 32)        1568        conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 4, 64)        6208        conv1d_1[0][0]                   
______________________________________________________________________________________________

In [8]:
def generate_true_samples(batch_size):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    x, true_labels = x_train[idx], y_train[idx]
    return x,true_labels

def getNoise():
    return  np.random.normal(0, 1, (batch_size, z_input_size))
def generate_fake_samples(generator,batch_size):
    noise = getNoise()
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    x = x_train[idx]
    fake_labels = generator.predict([noise, x])
    return x, fake_labels

In [9]:
n_epochs=10
batch_size=4

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))
    
for epoch in range(n_epochs):
    print("e:",epoch)
    for batch_idx in range(int(x_train.shape[0] // batch_size)):
        #print("b:",batch_idx)
        x,true_labels = generate_true_samples(batch_size)
        d_loss_real = discriminator.train_on_batch([x, true_labels], valid)
        
        noise = getNoise()
        fake_labels = generator.predict([noise, x])
        
        #print(x,true_labels,fake_labels)
        
        d_loss_fake = discriminator.train_on_batch([x, fake_labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        



        x2,_ = generate_true_samples(batch_size)
        g_loss = combined.train_on_batch([noise, x2], valid)
    
    pf = discriminator.predict([x, fake_labels])        
    pt = discriminator.predict([x, true_labels])
    pc = combined.predict([noise, x2])

    print('fake:',pf)    
    print('true:',pt)    
    print('combined:',pc)
    print(epoch, " of ", n_epochs)
    print("g_loss:",g_loss)
    print("d_loss:",d_loss)

e: 0
fake: [[0.4679103 ]
 [0.48215315]
 [0.44571957]
 [0.45805982]]
true: [[0.48676288]
 [0.4679103 ]
 [0.48676288]
 [0.48676288]]
combined: [[0.4679103]
 [0.4847887]
 [0.4456266]
 [0.4582684]]
0  of  10
g_loss: 0.7694499492645264
d_loss: [0.68390971 0.5       ]
e: 1
fake: [[0.45836037]
 [0.40266085]
 [0.45836037]
 [0.45836037]]
true: [[0.73743725]
 [0.4807279 ]
 [0.4033912 ]
 [0.5312338 ]]
combined: [[0.47312763]
 [0.40346462]
 [0.47312763]
 [0.47312763]]
1  of  10
g_loss: 0.8125012516975403
d_loss: [0.63941273 0.75      ]
e: 2
fake: [[0.20097288]
 [0.30131096]
 [0.34184867]
 [0.4934702 ]]
true: [[0.20097288]
 [0.9013509 ]
 [0.9013509 ]
 [0.9013509 ]]
combined: [[0.20097288]
 [0.8012929 ]
 [0.8515787 ]
 [0.49843347]]
2  of  10
g_loss: 1.146985411643982
d_loss: [0.4955094 0.75     ]
e: 3
fake: [[0.4688006 ]
 [0.25987887]
 [0.25987887]
 [0.25987887]]
true: [[0.8729364]
 [0.8729364]
 [0.8729364]
 [0.8729364]]
combined: [[0.6415047 ]
 [0.25987887]
 [0.25987887]
 [0.25987887]]
3  of  10
g_

In [10]:
from sklearn.metrics import confusion_matrix

noise = np.random.normal(0, 1, (x_val.shape[0], z_input_size))
yhat = generator.predict([noise, x_val])
        
yhat=scaler.inverse_transform(yhat).flatten()
y_val0=scaler.inverse_transform(y_val).flatten()
print(y_val)
print(yhat)

file = open('prediction.csv', 'w')
file.write('ground truth\tprediction\n')
for i in range(0,len(y_val)):
    file.write(str(y_val0[i])+"\t"+str(yhat[i])+"\n")
file.close()

print("RMSE")
error=tf.keras.metrics.mean_squared_error(
    y_val0, yhat
)
print(np.sqrt(error))

[[0.0141186 ]
 [0.        ]
 [0.        ]
 ...
 [0.07686223]
 [0.        ]
 [0.81869012]]
[ 37.39542    74.70058    57.243263  ...   1.2468143 155.71715
  78.58072  ]
RMSE
104.867096
